**OS path**
- 학원 노트북용 --> C:/Users/Playdata/Playdata_anl/1st_Project/dataset/
- 개인 노트북용 --> C:/Users/Brian/Desktop/playdata/Playdata_anl/1st_Project/dataset/

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
%matplotlib inline

# 이 두 줄의 코드는 matplotlib의 기본 scheme말고, seaborn scheme을 세팅해준다
# 일일이 graph의 font size를 지정할 필요 없이, seaborn의 font_scale을 사용하면 편리하다
plt.style.use('seaborn')
sns.set(font_scale = 2.5)

# 그래프에서 한글 폰트 깨지는 문제를 해결해주기 위한 코드
from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

import platform

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname = path).get_name()
    rc('font', family = font_name)
else:
    print('Unknown system... sorry~~~~') 

In [2]:
final_data = pd.read_csv("C:/Users/Playdata/Playdata_anl/1st_Project/dataset/final_data.csv")
final_data.drop(['Unnamed: 0'], axis = 1, inplace = True)
final_data

,date,line,station_num,station_name,boarding,subway_board_5,subway_board_6,subway_board_7,subway_board_8,subway_board_9,...,total_20,total_21,total_22,total_23,total_5,total_6,total_7,total_8,total_9,weekdays
0,2017-01-01,1호선,150,서울역,승차,470,286,397,786,1421,...,14464.0,15163.0,16337.0,16005.0,16668.0,16699.0,15777.0,14371.0,13411.0,6
1,2017-01-01,1호선,150,서울역,하차,278,880,859,964,1407,...,14464.0,15163.0,16337.0,16005.0,16668.0,16699.0,15777.0,14371.0,13411.0,6
2,2017-01-01,6호선,2629,삼각지,하차,36,72,53,99,134,...,20936.0,20835.0,21175.0,20946.0,20677.0,20328.0,19835.0,20299.0,21139.0,6
3,2017-01-01,6호선,2629,삼각지,승차,58,79,45,75,102,...,20936.0,20835.0,21175.0,20946.0,20677.0,20328.0,19835.0,20299.0,21139.0,6
4,2017-01-01,6호선,2628,효창공원앞,하차,24,92,79,94,169,...,40697.0,35855.0,33074.0,25661.0,28419.0,30523.0,32648.0,36860.0,41269.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560689,2019-12-31,3호선,341,경찰병원,하차,13,108,253,1201,580,...,13562.0,13940.0,14566.0,16078.0,16341.0,16114.0,16411.0,15534.0,15049.0,1
560690,2019-12-31,3호선,342,오금,하차,0,61,167,580,187,...,25061.0,26537.0,27781.0,28352.0,28091.0,28228.0,28606.0,28500.0,28682.0,1
560691,2019-12-31,3호선,342,오금,승차,120,243,922,1182,534,...,25061.0,26537.0,27781.0,28352.0,28091.0,28228.0,28606.0,28500.0,28682.0,1
560692,2019-12-31,3호선,321,충무로,승차,0,0,0,0,0,...,16935.0,17991.0,18136.0,19083.0,19899.0,19387.0,17741.0,16465.0,15458.0,1


## 파생 변수 생성
### date(날짜) 변수로부터 month(월) 변수를 생성

In [3]:
import datetime as dt

final_data['date'] = pd.to_datetime(final_data['date'], format = '%Y-%m-%d')

final_data['month'] = final_data['date'].dt.month
final_data['month']

0          1
1          1
2          1
3          1
4          1
          ..
560689    12
560690    12
560691    12
560692    12
560693    12
Name: month, Length: 560694, dtype: int64

### month(월) 변수를 사용하여 season(계절) 변수를 생성

In [4]:
def category_month(x):
    if x >= 3 and x <= 5:
        return "spring"
    elif x >= 6 and x <= 8:
        return "summer"
    elif x >= 9 and x <= 11:
        return "autumn"
    else:
        return "winter"
    
final_data['season'] = final_data['month'].apply(category_month)
final_data['season'].value_counts()

spring    141312
summer    141312
autumn    139776
winter    138294
Name: season, dtype: int64

## 변수명 변경
### 모델에 넣어줄 "기온", "습도" 변수명을 영어로 변경
1. **기온_0 + 기온_5~23**
    - temp_0 + temp_5~23 으로 변수명 변경해주어야 함
2. **습도(%)_0 + 습도(%)_5~23**
    - humidity_0 + humidity_5~23 으로 변수명 변경해주어야 함
3. **강수량(mm)_0 + 강수량(mm)_5~23**
    - rain_0 + rain_5~23 으로 변수명 변경해주어야 함

In [5]:
from tqdm.notebook import tqdm

for i in tqdm(list(range(5, 24)) + [0]):
    final_data.rename(columns = {f'기온_{i}':f'temp_{i}'}, inplace = True)
    final_data.rename(columns = {f'습도(%)_{i}':f'humidity_{i}'}, inplace = True)
    final_data.rename(columns = {f'강수량(mm)_{i}':f'rain_{i}'}, inplace = True)

In [6]:
# "기온" 변수의 변수명이 잘 변경되었는지 확인
final_data.iloc[0:2, 47:67]

,temp_0,temp_5,temp_6,temp_7,temp_8,temp_9,temp_10,temp_11,temp_12,temp_13,temp_14,temp_15,temp_16,temp_17,temp_18,temp_19,temp_20,temp_21,temp_22,temp_23
0,1.0,1.4,0.1,1.5,1.9,2.1,2.9,3.0,4.4,4.3,6.6,6.4,6.5,6.0,5.0,4.6,4.2,3.8,3.8,4.2
1,1.0,1.4,0.1,1.5,1.9,2.1,2.9,3.0,4.4,4.3,6.6,6.4,6.5,6.0,5.0,4.6,4.2,3.8,3.8,4.2


In [7]:
# "습도" 변수의 변수명이 잘 변경되었는지 확인
final_data.iloc[0:2, 87:107]

,humidity_0,humidity_5,humidity_6,humidity_7,humidity_8,humidity_9,humidity_10,humidity_11,humidity_12,humidity_13,humidity_14,humidity_15,humidity_16,humidity_17,humidity_18,humidity_19,humidity_20,humidity_21,humidity_22,humidity_23
0,79.0,86.0,87.0,87.0,87.0,83.0,77.0,71.0,69.0,65.0,61.0,65.0,67.0,69.0,70.0,73.0,73.0,74.0,75.0,75.0
1,79.0,86.0,87.0,87.0,87.0,83.0,77.0,71.0,69.0,65.0,61.0,65.0,67.0,69.0,70.0,73.0,73.0,74.0,75.0,75.0


In [8]:
# "강수량" 변수의 변수명이 잘 변경되었는지 확인
final_data.iloc[0:2, 147:167]

,rain_0,rain_5,rain_6,rain_7,rain_8,rain_9,rain_10,rain_11,rain_12,rain_13,rain_14,rain_15,rain_16,rain_17,rain_18,rain_19,rain_20,rain_21,rain_22,rain_23
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 혼잡도 지표 생성 후, 새로운 target 변수 생성
- 혼잡도: (승차 인원 + 하차 인원) * 100/(지하철 역 승강장 면적)
- 새로운 target 변수명: doc_0 + doc_5~23
    - doc: degree of congestion

**승장강 면적 변수가 들어있는 데이터 불러오기**

In [64]:
subway_platform_area = pd.read_csv("C:/Users/Playdata/Playdata_anl/1st_Project/dataset/subway/subway_platform_area.csv")
subway_platform_area

,line,station_name,platform_area
0,1,서울역,2080.00
1,1,시청,3784.00
2,1,종각,2672.81
3,1,종로3가,3683.00
4,1,종로5가,2208.00
...,...,...,...
253,8,문정,1915.20
254,8,장지,1628.15
255,8,복정,2141.55
256,3,충무로,3860.00


**final_data + subway_platform_area 데이터 결합**

- subway_platform_area 데이터 셋의 line 변수가 int로 들어가있다.
    - 이 부분을 "1호선" 이런 식으로 변경해주겠다.

In [65]:
subway_platform_area.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258 entries, 0 to 257
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   line           258 non-null    int64  
 1   station_name   258 non-null    object 
 2   platform_area  258 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.2+ KB


In [66]:
# subway_area_address 데이터 셋의 line 변수 뒤에 "호선"을 붙여주기
from tqdm.notebook import tqdm

for i in tqdm(range(len(subway_platform_area))):
    subway_platform_area['line'][i] = str(subway_platform_area['line'][i]) + "호선"
    
subway_platform_area['line'].value_counts()

C:\Users\Playdata\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Playdata\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


5호선    51
2호선    50
7호선    39
6호선    38
3호선    33
4호선    26
8호선    11
1호선    10
Name: line, dtype: int64

In [67]:
subway_platform_area.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258 entries, 0 to 257
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   line           258 non-null    object 
 1   station_name   258 non-null    object 
 2   platform_area  258 non-null    float64
dtypes: float64(1), object(2)
memory usage: 6.2+ KB


- 아래 부분은 final_data와 subway_platform_area의 inner join이 제대로 수행되지 않아서 확인용으로 만들어놓은 코드이다.
    - subway_platform_area 데이터 셋에서 "경기도" 및 "인천광역시"에 해당하는 지하철 역들은 제거해주었고, 6호선 "연신내"역과 3호선 "충무로"역은 따로 제거해주지 않았다.

In [56]:
# final_data['check_var1'] = final_data['line'] + "_" + final_data['station_name']
# subway_platform_area['check_var2'] = subway_platform_area['line'] + "_" + subway_platform_area['station_name']

In [57]:
# final_data['check_var1']

0           1호선_서울역
1           1호선_서울역
2           6호선_삼각지
3           6호선_삼각지
4         6호선_효창공원앞
            ...    
560689     3호선_경찰병원
560690       3호선_오금
560691       3호선_오금
560692      3호선_충무로
560693      3호선_충무로
Name: check_var1, Length: 560694, dtype: object

In [58]:
# subway_platform_area['check_var2']

0       1호선_서울역
1        1호선_시청
2        1호선_종각
3      1호선_종로3가
4      1호선_종로5가
         ...   
251      8호선_송파
252    8호선_가락시장
253      8호선_문정
254      8호선_장지
255      8호선_복정
Name: check_var2, Length: 256, dtype: object

In [63]:
# result = []
# for i in tqdm(list(set(final_data['check_var1']))):
#     if i not in list(subway_platform_area['check_var2']):
#         result.append(i)
# result

['6호선_연신내', '3호선_충무로']

In [68]:
final_data2 = pd.merge(final_data, subway_platform_area, how = 'inner', on = ['station_name', 'line'])
final_data2

,date,line,station_num,station_name,boarding,subway_board_5,subway_board_6,subway_board_7,subway_board_8,subway_board_9,...,total_5,total_6,total_7,total_8,total_9,weekdays,month,season,check_var1,platform_area
0,2017-01-01,1호선,150,서울역,승차,470,286,397,786,1421,...,16668.0,16699.0,15777.0,14371.0,13411.0,6,1,winter,1호선_서울역,2080.00
1,2017-01-01,1호선,150,서울역,하차,278,880,859,964,1407,...,16668.0,16699.0,15777.0,14371.0,13411.0,6,1,winter,1호선_서울역,2080.00
2,2017-01-02,1호선,150,서울역,하차,356,2366,4852,9813,4784,...,32492.0,31454.0,29285.0,25878.0,24702.0,0,1,winter,1호선_서울역,2080.00
3,2017-01-02,1호선,150,서울역,승차,411,536,2297,3665,2103,...,32492.0,31454.0,29285.0,25878.0,24702.0,0,1,winter,1호선_서울역,2080.00
4,2017-01-03,1호선,150,서울역,하차,303,1719,4452,10406,4863,...,47121.0,47685.0,47704.0,45729.0,44860.0,1,1,winter,1호선_서울역,2080.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560689,2019-12-26,6호선,2615,연신내,승차,0,0,0,0,0,...,18455.0,17957.0,19156.0,16839.0,15768.0,3,12,winter,6호선_연신내,2419.54
560690,2019-12-27,6호선,2615,연신내,승차,0,0,0,0,0,...,32948.0,34038.0,39395.0,38533.0,37710.0,4,12,winter,6호선_연신내,2419.54
560691,2019-12-27,6호선,2615,연신내,하차,0,0,0,0,0,...,32948.0,34038.0,39395.0,38533.0,37710.0,4,12,winter,6호선_연신내,2419.54
560692,2019-12-28,6호선,2615,연신내,하차,0,0,0,0,0,...,24080.0,24251.0,26300.0,24654.0,23424.0,5,12,winter,6호선_연신내,2419.54


In [69]:
len(final_data)

560694

- 새롭게 합쳐준 final_data2 데이터 셋에서 "혼잡도" 지표를 계산한 새로운 변수를 생성해주도록 하겠다.

In [70]:
column_names = final_data2.columns.tolist()
for idx, val in enumerate(column_names):
    print(f'{idx}. {val}')

0. date
1. line
2. station_num
3. station_name
4. boarding
5. subway_board_5
6. subway_board_6
7. subway_board_7
8. subway_board_8
9. subway_board_9
10. subway_board_10
11. subway_board_11
12. subway_board_12
13. subway_board_13
14. subway_board_14
15. subway_board_15
16. subway_board_16
17. subway_board_17
18. subway_board_18
19. subway_board_19
20. subway_board_20
21. subway_board_21
22. subway_board_22
23. subway_board_23
24. subway_board_0
25. total
26. type
27. area
28. gu
29. address
30. transfer
31. transfer_num
32. entrance
33. gate
34. nsi_cnt
35. hsi_cnt
36. msi_cnt
37. uvi_cnt
38. hti_cnt
39. mti_cnt
40. dong
41. pop_join_key
42. weather_join_key
43. 지점명
44. 주소
45. 구
46. 날짜
47. temp_0
48. temp_5
49. temp_6
50. temp_7
51. temp_8
52. temp_9
53. temp_10
54. temp_11
55. temp_12
56. temp_13
57. temp_14
58. temp_15
59. temp_16
60. temp_17
61. temp_18
62. temp_19
63. temp_20
64. temp_21
65. temp_22
66. temp_23
67. 풍속(m/s)_0
68. 풍속(m/s)_5
69. 풍속(m/s)_6
70. 풍속(m/s)_7
71. 풍속(m/s)_8
72

In [71]:
times = final_data2.iloc[:, 5:25].columns.tolist()
len(times)

20

In [72]:
from tqdm.notebook import tqdm

degree_of_congestion = []
for time in tqdm(times):
    temp = []
    for idx in tqdm(range(0, len(final_data2), 2)):
        temp.append(round((final_data2.loc[idx, time]+final_data2.loc[idx+1, time])/final_data2.loc[idx, 'platform_area']*100, 1))
        temp.append(round((final_data2.loc[idx, time]+final_data2.loc[idx+1, time])/final_data2.loc[idx, 'platform_area']*100, 1))
    degree_of_congestion.append(temp)
len(degree_of_congestion)

20

In [73]:
# list to dataframe
final_data3 = pd.DataFrame(degree_of_congestion, index = times).T
final_data3

,subway_board_5,subway_board_6,subway_board_7,subway_board_8,subway_board_9,subway_board_10,subway_board_11,subway_board_12,subway_board_13,subway_board_14,subway_board_15,subway_board_16,subway_board_17,subway_board_18,subway_board_19,subway_board_20,subway_board_21,subway_board_22,subway_board_23,subway_board_0
0,36.0,56.1,60.4,84.1,136.0,148.1,206.9,236.6,262.9,236.1,281.7,305.4,333.3,282.1,264.6,208.4,266.0,174.1,70.6,14.0
1,36.0,56.1,60.4,84.1,136.0,148.1,206.9,236.6,262.9,236.1,281.7,305.4,333.3,282.1,264.6,208.4,266.0,174.1,70.6,14.0
2,36.9,139.5,343.7,648.0,331.1,226.0,259.2,280.2,271.7,233.4,308.7,315.7,365.4,579.8,351.0,247.3,233.1,148.3,80.6,19.1
3,36.9,139.5,343.7,648.0,331.1,226.0,259.2,280.2,271.7,233.4,308.7,315.7,365.4,579.8,351.0,247.3,233.1,148.3,80.6,19.1
4,34.6,106.2,288.9,651.0,327.5,217.4,256.4,254.9,270.9,233.4,288.6,310.3,340.0,598.8,353.4,239.5,230.1,153.2,81.9,21.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560689,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
560690,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
560691,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
560692,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [74]:
# 계산된 혼잡도로 기존 데이터 셋을 변환
for i in times:
    final_data2[i] = final_data3[i]

In [75]:
# 혼잡도 변수명 변경
for i in list(range(5, 24)) + [0]:
    final_data2.rename(columns = {f"subway_board_{i}":f"doc_{i}"}, inplace = True)

In [76]:
# '하차'인 경우를 drop
drop_index = final_data2[final_data2['boarding'] == '하차'].index
final_data2 = final_data2.drop(drop_index)

In [79]:
# index 재설정 및 boarding 변수 제거
final_data2.reset_index(drop=True, inplace=True)
final_data2.drop('boarding', axis=1, inplace=True)
final_data2

,date,line,station_num,station_name,doc_5,doc_6,doc_7,doc_8,doc_9,doc_10,...,total_5,total_6,total_7,total_8,total_9,weekdays,month,season,check_var1,platform_area
0,2017-01-01,1호선,150,서울역,36.0,56.1,60.4,84.1,136.0,148.1,...,16668.0,16699.0,15777.0,14371.0,13411.0,6,1,winter,1호선_서울역,2080.00
1,2017-01-02,1호선,150,서울역,36.9,139.5,343.7,648.0,331.1,226.0,...,32492.0,31454.0,29285.0,25878.0,24702.0,0,1,winter,1호선_서울역,2080.00
2,2017-01-03,1호선,150,서울역,34.6,106.2,288.9,651.0,327.5,217.4,...,47121.0,47685.0,47704.0,45729.0,44860.0,1,1,winter,1호선_서울역,2080.00
3,2017-01-04,1호선,150,서울역,32.2,106.2,286.8,629.2,357.7,229.5,...,34082.0,33410.0,32893.0,31024.0,29672.0,2,1,winter,1호선_서울역,2080.00
4,2017-01-05,1호선,150,서울역,35.0,112.7,280.9,663.8,347.3,230.6,...,26377.0,25869.0,25479.0,24163.0,23095.0,3,1,winter,1호선_서울역,2080.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280342,2019-12-22,6호선,2615,연신내,0.0,0.0,0.0,0.0,0.0,0.0,...,22986.0,21622.0,19126.0,16908.0,15777.0,6,12,winter,6호선_연신내,2419.54
280343,2019-12-23,6호선,2615,연신내,0.0,0.0,0.0,0.0,0.0,0.0,...,18599.0,18175.0,18470.0,17199.0,16505.0,0,12,winter,6호선_연신내,2419.54
280344,2019-12-26,6호선,2615,연신내,0.0,0.0,0.0,0.0,0.0,0.0,...,18455.0,17957.0,19156.0,16839.0,15768.0,3,12,winter,6호선_연신내,2419.54
280345,2019-12-27,6호선,2615,연신내,0.0,0.0,0.0,0.0,0.0,0.0,...,32948.0,34038.0,39395.0,38533.0,37710.0,4,12,winter,6호선_연신내,2419.54


In [80]:
560694/2

280347.0

## 비 내림 여부를 의미하는 rain 변수 생성
- rain_0, rain_5~23 변수들의 경우, 0 값이 대부분이라서 정제하지 않은 채 feature로 사용하기엔 무리가 있어보인다.
- 따라서 하루 중, 어느 한 시간에서라도 비가 내렸으면 "1", 그렇지 않으면 "0"으로 구분해주는 새로운 rain 변수를 생성해서 사용하도록 하겠다.

In [81]:
# 안전하게 복사본을 만들어서 사용하자!!
final_data_copy = final_data2.copy()

In [82]:
final_data_copy.iloc[0, 146:166]

rain_0     0
rain_5     0
rain_6     0
rain_7     0
rain_8     0
rain_9     0
rain_10    0
rain_11    0
rain_12    0
rain_13    0
rain_14    0
rain_15    0
rain_16    0
rain_17    0
rain_18    0
rain_19    0
rain_20    0
rain_21    0
rain_22    0
rain_23    0
Name: 0, dtype: object

In [85]:
from tqdm.notebook import tqdm

# rain 변수 컬럼을 하나 미리 생성해두고, 0인 값들을 변경하는 식으로 for문을 작성
final_data_copy['rain'] = 0

for i in tqdm(range(len(final_data_copy))):
    if sum(final_data_copy.iloc[i, 146:166].values.tolist()) != 0:
        final_data_copy['rain'][i] = 1
    else:
        final_data_copy['rain'][i] = 0
    
final_data_copy['rain'].value_counts()

C:\Users\Playdata\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\Playdata\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


0    222545
1     57802
Name: rain, dtype: int64

- **fianl_data_copy 데이터 셋의 rain 변수**만 가져와서 **final_data2에 ```concat()```**

In [86]:
df_rain = final_data_copy[['rain']]
df_rain

,rain
0,0
1,0
2,0
3,0
4,0
...,...
280342,0
280343,0
280344,0
280345,0


In [87]:
total_dataset = pd.concat([final_data2, df_rain], axis = 1)
total_dataset

,date,line,station_num,station_name,doc_5,doc_6,doc_7,doc_8,doc_9,doc_10,...,total_6,total_7,total_8,total_9,weekdays,month,season,check_var1,platform_area,rain
0,2017-01-01,1호선,150,서울역,36.0,56.1,60.4,84.1,136.0,148.1,...,16699.0,15777.0,14371.0,13411.0,6,1,winter,1호선_서울역,2080.00,0
1,2017-01-02,1호선,150,서울역,36.9,139.5,343.7,648.0,331.1,226.0,...,31454.0,29285.0,25878.0,24702.0,0,1,winter,1호선_서울역,2080.00,0
2,2017-01-03,1호선,150,서울역,34.6,106.2,288.9,651.0,327.5,217.4,...,47685.0,47704.0,45729.0,44860.0,1,1,winter,1호선_서울역,2080.00,0
3,2017-01-04,1호선,150,서울역,32.2,106.2,286.8,629.2,357.7,229.5,...,33410.0,32893.0,31024.0,29672.0,2,1,winter,1호선_서울역,2080.00,0
4,2017-01-05,1호선,150,서울역,35.0,112.7,280.9,663.8,347.3,230.6,...,25869.0,25479.0,24163.0,23095.0,3,1,winter,1호선_서울역,2080.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280342,2019-12-22,6호선,2615,연신내,0.0,0.0,0.0,0.0,0.0,0.0,...,21622.0,19126.0,16908.0,15777.0,6,12,winter,6호선_연신내,2419.54,0
280343,2019-12-23,6호선,2615,연신내,0.0,0.0,0.0,0.0,0.0,0.0,...,18175.0,18470.0,17199.0,16505.0,0,12,winter,6호선_연신내,2419.54,0
280344,2019-12-26,6호선,2615,연신내,0.0,0.0,0.0,0.0,0.0,0.0,...,17957.0,19156.0,16839.0,15768.0,3,12,winter,6호선_연신내,2419.54,0
280345,2019-12-27,6호선,2615,연신내,0.0,0.0,0.0,0.0,0.0,0.0,...,34038.0,39395.0,38533.0,37710.0,4,12,winter,6호선_연신내,2419.54,0


## 분석에 필요한 변수들만 선택
**<Target 변수(혼잡도)> - 총 20개**
- doc_0 + doc_5~23
- doc: degree of congestion


**<범주형 변수> - 총 10개**
1. date (구분을 위해 넣어주었을 뿐, 사용 x)
    - 날짜
2. date에서 파생변수 생성
    - month(월)
    - weekdays(요일) --> 0: 월요일, ... , 6: 일요일
    - season(계절) (winter: 12 ~ 2월, spring: 3 ~ 5월, summer: 6 ~ 8월, autumn: 9 ~ 11월)
3. line
    - 1 ~ 8호선
4. station_num (구분을 위해 넣어주었을 뿐, 사용 x)
    - 지하철 역 코드번호
5. station_name (구분을 위해 넣어주었을 뿐, 사용 x)
    - 지하철 역 이름
6. type
    - 섬식/상대식/복합식
7. gu
    - 25개의 자치구
8. rain
    - 비 내림 여부
    - 모든 시간대 중, 하나의 시간대라도 비가 왔으면 "1", 그렇지 않으면 "0"


**<연속형 변수> - 총 190개**
1. area (사용 x)
    - 역사 면적
2. platform_area (혼잡도 지표를 계산할 때만 사용하고, 모델에는 사용 x)
    - 승강장 면적
3. transfer_num
    - 환승 노선 개수
4. entrance
    - 출입구 개수
5. nsi_cnt
    - 역 주변 시설 총 개수
6. hsi_cnt
    - 역 주변 고등학교 개수
7. msi_cnt
    - 역 주변 중학교 개수
8. uvi_cnt
    - 역 주변 대학교 개수
9. hti_cnt
    - 역 주변 병원 개수
10. mti_cnt
    - 역 주변 영화관 개수
11. 기온_0 + 기온_5~23
    - temp_0 + temp_5~23 으로 변수명 변경해주어야 함
12. 습도(%)_0 + 습도(%)_5~23
    - humidity_0 + humidity_5~23 으로 변수명 변경해주어야 함
13. 강수량(mm)_0 + 강수량(mm)_5~23 (사용 x)
    - rain_0 + rain_5~23 으로 변수명 변경해주어야 함
    - 이 변수들을 그냥 사용하는 것은 무의미해 보임
    - 따라서 비가 오는지 여부에 따라 0과 1로 구분해주는 새로운 범주형 "rain" 변수를 생성 해주었음
14. below_20_0 + below_20_5~23
    - 역이 속한 행정동의 시간대별 20세미만 유동인구 수
15. pop_20_0 + pop_20_5~23
    - 역이 속한 행정동의 시간대별 20대 유동인구 수
16. pop_30_0 + pop_30_5~23
    - 역이 속한 행정동의 시간대별 30대 유동인구 수
17. pop_40_0 + pop_40_5~23
    - 역이 속한 행정동의 시간대별 40대 유동인구 수
18. pop_50_0 + pop_50_5~23
    - 역이 속한 행정동의 시간대별 50대 유동인구 수
19. pop_60_0 + pop_60_5~23
    - 역이 속한 행정동의 시간대별 60대 유동인구 수
20. over_70_0 + over_70_5~23
    - 역시 속한 행정동의 시간대별 70세이상 유동인구 수

In [88]:
total_dataset2 = total_dataset[[# 범주형 변수
                                'date', 'month', 'weekdays', 'season', 'line', 'station_num', 'station_name', 'type', 'gu', 'rain',

                                # 연속형 변수
                                'area', 'platform_area', 'transfer_num', 'entrance', 'nsi_cnt', 'hsi_cnt', 'msi_cnt', 'uvi_cnt', 'hti_cnt', 'mti_cnt',

                                'temp_0', 'temp_5', 'temp_6', 'temp_7', 'temp_8', 'temp_9', 'temp_10',
                                'temp_11', 'temp_12', 'temp_13', 'temp_14', 'temp_15', 'temp_16', 'temp_17',
                                'temp_18', 'temp_19', 'temp_20', 'temp_21', 'temp_22', 'temp_23',

                                'humidity_0', 'humidity_5', 'humidity_6', 'humidity_7', 'humidity_8', 'humidity_9', 'humidity_10',
                                'humidity_11', 'humidity_12', 'humidity_13', 'humidity_14', 'humidity_15', 'humidity_16', 'humidity_17',
                                'humidity_18', 'humidity_19', 'humidity_20', 'humidity_21', 'humidity_22', 'humidity_23',

                                #'rain_0', 'rain_5', 'rain_6', 'rain_7', 'rain_8', 'rain_9', 'rain_10',
                                #'rain_11', 'rain_12', 'rain_13', 'rain_14', 'rain_15', 'rain_16', 'rain_17',
                                #'rain_18', 'rain_19', 'rain_20', 'rain_21', 'rain_22', 'rain_23',

                                'below_20_0', 'below_20_5', 'below_20_6', 'below_20_7', 'below_20_8', 'below_20_9', 'below_20_10',
                                'below_20_11', 'below_20_12', 'below_20_13', 'below_20_14', 'below_20_15', 'below_20_16', 'below_20_17',
                                'below_20_18', 'below_20_19', 'below_20_20', 'below_20_21', 'below_20_22', 'below_20_23',

                                'pop_20_0', 'pop_20_5', 'pop_20_6', 'pop_20_7', 'pop_20_8', 'pop_20_9', 'pop_20_10',
                                'pop_20_11', 'pop_20_12', 'pop_20_13', 'pop_20_14', 'pop_20_15', 'pop_20_16', 'pop_20_17',
                                'pop_20_18', 'pop_20_19', 'pop_20_20', 'pop_20_21', 'pop_20_22', 'pop_20_23',

                                'pop_30_0', 'pop_30_5', 'pop_30_6', 'pop_30_7', 'pop_30_8', 'pop_30_9', 'pop_30_10',
                                'pop_30_11', 'pop_30_12', 'pop_30_13', 'pop_30_14', 'pop_30_15', 'pop_30_16', 'pop_30_17',
                                'pop_30_18', 'pop_30_19', 'pop_30_20', 'pop_30_21', 'pop_30_22', 'pop_30_23',

                                'pop_40_0', 'pop_40_5', 'pop_40_6', 'pop_40_7', 'pop_40_8', 'pop_40_9', 'pop_40_10',
                                'pop_40_11', 'pop_40_12', 'pop_40_13', 'pop_40_14', 'pop_40_15', 'pop_40_16', 'pop_40_17',
                                'pop_40_18', 'pop_40_19', 'pop_40_20', 'pop_40_21', 'pop_40_22', 'pop_40_23',

                                'pop_50_0', 'pop_50_5', 'pop_50_6', 'pop_50_7', 'pop_50_8', 'pop_50_9', 'pop_50_10',
                                'pop_50_11', 'pop_50_12', 'pop_50_13', 'pop_50_14', 'pop_50_15', 'pop_50_16', 'pop_50_17',
                                'pop_50_18', 'pop_50_19', 'pop_50_20', 'pop_50_21', 'pop_50_22', 'pop_50_23',

                                'pop_60_0', 'pop_60_5', 'pop_60_6', 'pop_60_7', 'pop_60_8', 'pop_60_9', 'pop_60_10',
                                'pop_60_11', 'pop_60_12', 'pop_60_13', 'pop_60_14', 'pop_60_15', 'pop_60_16', 'pop_60_17',
                                'pop_60_18', 'pop_60_19', 'pop_60_20', 'pop_60_21', 'pop_60_22', 'pop_60_23',

                                'over_70_0', 'over_70_5', 'over_70_6', 'over_70_7', 'over_70_8', 'over_70_9', 'over_70_10',
                                'over_70_11', 'over_70_12', 'over_70_13', 'over_70_14', 'over_70_15', 'over_70_16', 'over_70_17',
                                'over_70_18', 'over_70_19', 'over_70_20', 'over_70_21', 'over_70_22', 'over_70_23',
                                
                                # Target 변수
                                'doc_0', 'doc_5', 'doc_6', 'doc_7', 'doc_8', 'doc_9', 'doc_10',
                                'doc_11', 'doc_12', 'doc_13', 'doc_14', 'doc_15', 'doc_16', 'doc_17',
                                'doc_18', 'doc_19', 'doc_20', 'doc_21', 'doc_22', 'doc_23']]

total_dataset2

,date,month,weekdays,season,line,station_num,station_name,type,gu,rain,...,doc_14,doc_15,doc_16,doc_17,doc_18,doc_19,doc_20,doc_21,doc_22,doc_23
0,2017-01-01,1,6,winter,1호선,150,서울역,섬식,중구,0,...,236.1,281.7,305.4,333.3,282.1,264.6,208.4,266.0,174.1,70.6
1,2017-01-02,1,0,winter,1호선,150,서울역,섬식,중구,0,...,233.4,308.7,315.7,365.4,579.8,351.0,247.3,233.1,148.3,80.6
2,2017-01-03,1,1,winter,1호선,150,서울역,섬식,중구,0,...,233.4,288.6,310.3,340.0,598.8,353.4,239.5,230.1,153.2,81.9
3,2017-01-04,1,2,winter,1호선,150,서울역,섬식,중구,0,...,245.4,277.8,310.4,376.0,623.5,344.5,235.8,218.9,161.1,85.7
4,2017-01-05,1,3,winter,1호선,150,서울역,섬식,중구,0,...,251.0,309.0,302.9,375.8,609.9,360.9,250.1,250.1,161.8,90.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280342,2019-12-22,12,6,winter,6호선,2615,연신내,상대식,은평구,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
280343,2019-12-23,12,0,winter,6호선,2615,연신내,상대식,은평구,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
280344,2019-12-26,12,3,winter,6호선,2615,연신내,상대식,은평구,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
280345,2019-12-27,12,4,winter,6호선,2615,연신내,상대식,은평구,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 결측치(인구 데이터 부분)가 존재하는 변수들을 포함하는 row들은 삭제
- 보다 나은 RMSE 값을 얻기 위해서는, 기존에 결측치였던 row들을 삭제해주는 것이 바람직하다.

In [89]:
len(total_dataset2[total_dataset2['below_20_5'].isnull()])

3072

In [90]:
total_dataset2.isnull().sum().sum()

430080

In [91]:
# 140이 나와야 함!!
430080/3072

140.0

In [92]:
total_dataset3 = total_dataset2.dropna()

In [93]:
total_dataset3.isnull().sum().sum()

0

In [94]:
# drop된 row의 개수는 3072개가 나와야 함!!
280347 - len(total_dataset3)

3072

- 위 결과를 보면, 결측치들이 존재하던 6144개의 row가 삭제되었음을 알 수 있다.

## 혼잡도가 0인 row들은 삭제
- **"충무로" 역**의 경우, **3호선에 해당하는 출입구(gate)가 없다.**
- **"연신내" 역**의 경우, **6호선에 해당하는 출입구(gate)가 없다.**
- 때문에 **모든 시간대의 지하철 승하차 인원 수가 0으로 기록**되어 있으므로, **제거해주도록 하겠다.**

In [95]:
drop_index_num1 = total_dataset3[(total_dataset3['station_name'] == '충무로') & (total_dataset3['line'] == '3호선')].index
total_dataset3 = total_dataset3.drop(drop_index_num1)

drop_index_num2 = total_dataset3[(total_dataset['station_name'] == '연신내') & (total_dataset3['line'] == '6호선')].index
total_dataset3 = total_dataset3.drop(drop_index_num2)

C:\Users\Playdata\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [96]:
total_dataset3[(total_dataset3['station_name'] == '충무로') & (total_dataset3['line'] == '3호선')]

,date,month,weekdays,season,line,station_num,station_name,type,gu,rain,...,doc_14,doc_15,doc_16,doc_17,doc_18,doc_19,doc_20,doc_21,doc_22,doc_23


In [97]:
total_dataset3[(total_dataset3['station_name'] == '연신내') & (total_dataset3['line'] == '6호선')]

,date,month,weekdays,season,line,station_num,station_name,type,gu,rain,...,doc_14,doc_15,doc_16,doc_17,doc_18,doc_19,doc_20,doc_21,doc_22,doc_23


### 모든 작업이 끝난 최종 데이터 셋을 csv 파일로 내보내기

In [98]:
total_dataset3.to_csv("C:/Users/Playdata/Playdata_anl/1st_Project/dataset/total_dataset.csv")